In [1]:
# Import statements
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math

In [2]:
# Convert the cleaned data to a data frame
raceDist = pd.read_csv("442FinalProject/filled_in_data.csv")
raceDist.head()

,location,state_pop,white_pop,black_pop,hispanic_pop,asian_pop,white_vacc,black_vacc,hispanic_vacc,asian_vacc
0,Alabama,4903185,0.654,0.265,0.044,0.014,0.71,0.24,0.0375,0.02
1,Alaska,731545,0.600,0.022,0.070,0.060,0.39,0.02,0.0500,0.05
2,Arizona,7278717,0.542,0.043,0.318,0.033,0.56,0.02,0.1400,0.04
3,Arkansas,3017804,0.721,0.152,0.078,0.016,0.82,0.12,0.2000,0.02
4,California,39512223,0.364,0.053,0.395,0.147,0.33,0.03,0.2500,0.15


In [3]:
# The goal is to obtain an IP weighted model for each race
# First, we need to obtain the outcome Y, which is the percentage of vaccinated individuals within a racial group
# Currently, we have the total racial distribution and the racial distribution within the vaccinated population
vaccines = pd.read_csv("442FinalProject/num_vaccines.csv")
vaccines.head()

,Title: COVID-19 Vaccines Delivered and Administered | KFF,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Location,Total COVID-19 Vaccines Delivered,Total COVID-19 Vaccines Administered,Share of Delivered Vaccines That Have Been Adm...,Number of People Who Have Been Vaccinated,Share of Population Vaccinated
1,United States,317128386,254229952,0.80166,148280788,0.45174568
2,Alabama,4042070,2559938,0.63332,1579093,0.32205454
3,Alaska,768205,569114,0.74084,308040,0.42108141
4,Arizona,6536895,5024348,0.76861,2943500,0.40439819


In [4]:
# We can find the percentage of vaccinated individuals within a racial group (Y) by using race_pop (the percentage 
# distribution of a race within the total state population), race_vacc (the percentage distribution of a race within the 
# vaccinated state population), and Share of Population Vaccinated (the percentage of all individuals who are vaccinated)
# Y = (race_vacc * Share of People Vaccinated) / race_pop
# We also want this value for all individuals who do not identify as the specified race
# We must find this for each race

# Convert vaccines column to useable data
vaccines = vaccines.drop(labels=[0, 1], axis=0)
vaccines = vaccines.drop(labels=range(53,78), axis=0)
vaccines = vaccines.reset_index(drop=True)
vaccines.iloc[:, 5] = vaccines.iloc[:, 5].astype('float64')

# White
white = pd.DataFrame()
white['white_vacc'] = (raceDist['white_vacc'] * vaccines.iloc[:, 5]) / raceDist['white_pop']
white['non_white_vacc'] = ((raceDist['black_vacc'] * vaccines.iloc[:, 5]) + (raceDist['hispanic_vacc'] * vaccines.iloc[:, 5]) + (raceDist['asian_vacc'] * vaccines.iloc[:, 5])) / (raceDist['black_pop'] + raceDist['hispanic_pop'] + raceDist['asian_pop'])

# Black
black = pd.DataFrame()
black['black_vacc'] = (raceDist['black_vacc'] * vaccines.iloc[:, 5]) / raceDist['black_pop']
black['non_black_vacc'] = ((raceDist['white_vacc'] * vaccines.iloc[:, 5]) + (raceDist['hispanic_vacc'] * vaccines.iloc[:, 5]) + (raceDist['asian_vacc'] * vaccines.iloc[:, 5])) / (raceDist['white_pop'] + raceDist['hispanic_pop'] + raceDist['asian_pop'])

# Hispanic
hispanic = pd.DataFrame()
hispanic['hispanic_vacc'] = (raceDist['hispanic_vacc'] * vaccines.iloc[:, 5]) / raceDist['hispanic_pop']
hispanic['non_hispanic_vacc'] = ((raceDist['white_vacc'] * vaccines.iloc[:, 5]) + (raceDist['black_vacc'] * vaccines.iloc[:, 5]) + (raceDist['asian_vacc'] * vaccines.iloc[:, 5])) / (raceDist['white_pop'] + raceDist['black_pop'] + raceDist['asian_pop'])

# Asian
asian = pd.DataFrame()
asian['asian_vacc'] = (raceDist['asian_vacc'] * vaccines.iloc[:, 5]) / raceDist['asian_pop']
asian['non_asian_vacc'] = ((raceDist['white_vacc'] * vaccines.iloc[:, 5]) + (raceDist['black_vacc'] * vaccines.iloc[:, 5]) + (raceDist['hispanic_vacc'] * vaccines.iloc[:, 5])) / (raceDist['white_pop'] + raceDist['black_pop'] + raceDist['hispanic_pop'])

white.head()

,white_vacc,non_white_vacc
0,0.349631,0.296629
1,0.273703,0.332433
2,0.417828,0.205278
3,0.402317,0.488915
4,0.437204,0.348516


In [5]:
# We will start by identifying the confounders and obtaining the relevant information for that
# The confounders that will be considered are: population density and vaccine availability
# First, we will find the population density

# Many states have losts of space that is completely uninhabited, where most of the people collect at certain locations
# This calculation will account for that by using the urban index for each state
# The urban index is the natural log of the average number of people living within a 5-mile radius of a given resident

# Get the urban indeces
confounders = pd.read_csv("442FinalProject/state_urbanization.csv")
confounders = confounders.drop(labels=[2, 12, 37, 42, 50], axis=0)
confounders = confounders.reset_index(drop=True)

# Get the state areas
areas = pd.read_csv("state-areas.csv")

# Get the average number of people living in 1 square mile of populated land
# This is the population density (accounting for uninhabited land)
confounders['urbanindex'] = np.exp(confounders['urbanindex'])
confounders['urbanindex'] = confounders['urbanindex'] / (25 * math.pi)
confounders.head()

,state,urbanindex
0,Alabama,189.110082
1,Alaska,79.230259
2,Arizona,1028.754668
3,Arkansas,133.732066
4,California,2506.579338


In [6]:
# The next confounder is vaccine availability
# This will be Total Vaccines Delivered / State Population
# A higher number suggests that vaccines are more available

vaccines.iloc[:, 1] = vaccines.iloc[:, 1].astype('float64')
confounders['vacc_avail'] = vaccines.iloc[:, 1] / raceDist['state_pop']
confounders.head()

,state,urbanindex,vacc_avail
0,Alabama,189.110082,0.824376
1,Alaska,79.230259,1.050113
2,Arizona,1028.754668,0.898083
3,Arkansas,133.732066,0.870984
4,California,2506.579338,0.975960


In [7]:
# Add confounders as columns in the race dataframes

# White
white['pop'] = raceDist['white_pop']
white['pop_density'] = confounders['urbanindex']
white['vacc_avail'] = confounders['vacc_avail']
white.insert(0, 'state', raceDist['location'])

# Black
black['pop'] = raceDist['black_pop']
black['pop_density'] = confounders['urbanindex']
black['vacc_avail'] = confounders['vacc_avail']
black.insert(0, 'state', raceDist['location'])

# Hispanic
hispanic['pop'] = raceDist['hispanic_pop']
hispanic['pop_density'] = confounders['urbanindex']
hispanic['vacc_avail'] = confounders['vacc_avail']
hispanic.insert(0, 'state', raceDist['location'])

# Asian
asian['pop'] = raceDist['asian_pop']
asian['pop_density'] = confounders['urbanindex']
asian['vacc_avail'] = confounders['vacc_avail']
asian.insert(0, 'state', raceDist['location'])

white.head()

,state,white_vacc,non_white_vacc,pop,pop_density,vacc_avail
0,Alabama,0.349631,0.296629,0.654,189.110082,0.824376
1,Alaska,0.273703,0.332433,0.600,79.230259,1.050113
2,Arizona,0.417828,0.205278,0.542,1028.754668,0.898083
3,Arkansas,0.402317,0.488915,0.721,133.732066,0.870984
4,California,0.437204,0.348516,0.364,2506.579338,0.975960


In [8]:
# Next, we will parametrically estimate the IP weights to be applied to the outcome
# First, we need to separate the states into separate bins depending on their covariates
# The bins will be:
#       population density:   x < 100, 100 <= x < 1000, 1000 <= x
#       vaccine availability: x < 1,   1 <= x
#       racial distribution:
#             white:    x < .55, .55 <= x < .75, .75 <= x
#             black:    x < .15, .15 <= x
#             hispanic: x < .10, .10 <= x
#             asian: N/A

# For the sake of this analysis, the treatment variable will be considered to be binary, where A = race = 1 and A = not race 
# = 0
# race_vacc is the continous outcome for A = 1 and not_race_vacc is the continous outcome for A = 0

# Restructure the dataframes for regression

# White
#temp = white.append(white)
#temp.insert(1, 'outcome', white['white_vacc'].append(white['non_white_vacc']))
#list1 = np.ones((51,), dtype=int)
#list2 = np.zeros((51,), dtype=int)
#temp.insert(1, 'treatment', np.concatenate((list1, list2)))
#temp = temp.drop(columns=['white_vacc', 'non_white_vacc'])
#white = temp

# Black
#temp = black.append(black)
#temp.insert(1, 'outcome', black['black_vacc'].append(black['non_black_vacc']))
#list1 = np.ones((51,), dtype=int)
#list2 = np.zeros((51,), dtype=int)
#temp.insert(1, 'treatment', np.concatenate((list1, list2)))
#temp = temp.drop(columns=['black_vacc', 'non_black_vacc'])
#black = temp

# Hispanic
#temp = hispanic.append(hispanic)
#temp.insert(1, 'outcome', hispanic['hispanic_vacc'].append(hispanic['non_hispanic_vacc']))
#list1 = np.ones((51,), dtype=int)
#list2 = np.zeros((51,), dtype=int)
#temp.insert(1, 'treatment', np.concatenate((list1, list2)))
#temp = temp.drop(columns=['hispanic_vacc', 'non_hispanic_vacc'])
#hispanic = temp

# Asian
#temp = asian.append(asian)
#temp.insert(1, 'outcome', asian['asian_vacc'].append(asian['non_asian_vacc']))
#list1 = np.ones((51,), dtype=int)
#list2 = np.zeros((51,), dtype=int)
#temp.insert(1, 'treatment', np.concatenate((list1, list2)))
#temp = temp.drop(columns=['asian_vacc', 'non_asian_vacc'])
#asian = temp

white.head()

,state,white_vacc,non_white_vacc,pop,pop_density,vacc_avail
0,Alabama,0.349631,0.296629,0.654,189.110082,0.824376
1,Alaska,0.273703,0.332433,0.600,79.230259,1.050113
2,Arizona,0.417828,0.205278,0.542,1028.754668,0.898083
3,Arkansas,0.402317,0.488915,0.721,133.732066,0.870984
4,California,0.437204,0.348516,0.364,2506.579338,0.975960


In [9]:
# Find the propensity scores
# Since the propensity scores can't be computed on the individual level, the racial distribution within the total population
# will be used as the treatment variable

# White
model_string = 'pop ~ pop_density + vacc_avail'
white_model = smf.glm(model_string, data=white, family=sm.families.Binomial()).fit()
white['prop_score'] = white_model.predict(white)

# Black
black_model = smf.glm(model_string, data=black, family=sm.families.Binomial()).fit()
black['prop_score'] = black_model.predict(black)

# Hispanic
hispanic_model = smf.glm(model_string, data=hispanic, family=sm.families.Binomial()).fit()
hispanic['prop_score'] = hispanic_model.predict(hispanic)

# Asian
asian_model = smf.glm(model_string, data=asian, family=sm.families.Binomial()).fit()
asian['prop_score'] = asian_model.predict(asian)

asian.head()

,state,asian_vacc,non_asian_vacc,pop,pop_density,vacc_avail,prop_score
0,Alabama,0.460078,0.330248,0.014,189.110082,0.824376,0.027054
1,Alaska,0.350901,0.279910,0.060,79.230259,1.050113,0.064576
2,Arizona,0.490180,0.322444,0.033,1028.754668,0.898083,0.036846
3,Arkansas,0.442181,0.424047,0.016,133.732066,0.870984,0.032435
4,California,0.492091,0.362281,0.147,2506.579338,0.975960,0.051588


In [10]:
# The stabilized IP weights can be computed by using 'pop' and 'prop_score'
# 'pop' represents the percentage of individuals of the total population that belong to a specified race (aka the propability 
# of being in the treated group)
# The stabilized IP weight is pop / prop_score

# White
white['SW'] = white['pop'] / white['prop_score']

# Black
black['SW'] = black['pop'] / black['prop_score']

# Hispanic
hispanic['SW'] = hispanic['pop'] / hispanic['prop_score']

# Asian
asian['SW'] = asian['pop'] / asian['prop_score']

white.head()

,state,white_vacc,non_white_vacc,pop,pop_density,vacc_avail,prop_score,SW
0,Alabama,0.349631,0.296629,0.654,189.110082,0.824376,0.713036,0.917205
1,Alaska,0.273703,0.332433,0.600,79.230259,1.050113,0.726210,0.826207
2,Arizona,0.417828,0.205278,0.542,1028.754668,0.898083,0.667630,0.811827
3,Arkansas,0.402317,0.488915,0.721,133.732066,0.870984,0.717561,1.004792
4,California,0.437204,0.348516,0.364,2506.579338,0.975960,0.578076,0.629675


In [13]:
# Fit the MSM for each race

# White
white_MSM = smf.wls('white_vacc ~ pop', data=white, weights=np.array(white['SW'], dtype=np.float64)).fit()
white_ACE0 = white_MSM.params[0]
white_ACE1 = white_MSM.params[0] + white_MSM.params[1]
white_ACE = white_MSM.params[1]

# Black
black_MSM = smf.wls('black_vacc ~ pop', data=black, weights=np.array(black['SW'], dtype=np.float64)).fit()
black_ACE0 = black_MSM.params[0]
black_ACE1 = black_MSM.params[0] + black_MSM.params[1]
black_ACE = black_MSM.params[1]

# Hispanic
hispanic_MSM = smf.wls('hispanic_vacc ~ pop', data=hispanic, weights=np.array(hispanic['SW'], dtype=np.float64)).fit()
hispanic_ACE0 = hispanic_MSM.params[0]
hispanic_ACE1 = hispanic_MSM.params[0] + hispanic_MSM.params[1]
hispanic_ACE = hispanic_MSM.params[1]

# Asian
asian_MSM = smf.wls('asian_vacc ~ pop', data=asian, weights=np.array(asian['SW'], dtype=np.float64)).fit()
asian_ACE0 = asian_MSM.params[0]
asian_ACE1 = asian_MSM.params[0] + asian_MSM.params[1]
asian_ACE = asian_MSM.params[1]

In [22]:
# Print the ACEs

print("P(Y^(A = White) = 1) =    ", round(white_ACE1, 3), "\t P(Y^(A = White) = 0) =    ", round(white_ACE0, 3), "\t ACE = ", round(white_ACE, 3))
print("P(Y^(A = Black) = 1) =    ", round(black_ACE1, 3), "\t P(Y^(A = Black) = 0) =    ", round(black_ACE0, 3), "\t ACE = ", round(black_ACE, 3))
print("P(Y^(A = Hispanic) = 1) = ", round(hispanic_ACE1, 3), "\t P(Y^(A = Hispanic) = 0) = ", round(hispanic_ACE0, 3), "\t ACE =  ", round(hispanic_ACE, 3))
print("P(Y^(A = Asian) = 1) =    ", round(asian_ACE1, 3), "\t P(Y^(A = Asian) = 0) =    ", round(asian_ACE0, 3), "\t ACE =  ", round(asian_ACE, 3))

P(Y^(A = White) = 1) =     0.434 	 P(Y^(A = White) = 0) =     0.57 	 ACE =  -0.136
P(Y^(A = Black) = 1) =     0.273 	 P(Y^(A = Black) = 0) =     0.305 	 ACE =  -0.032
P(Y^(A = Hispanic) = 1) =  0.333 	 P(Y^(A = Hispanic) = 0) =  0.303 	 ACE =   0.03
P(Y^(A = Asian) = 1) =     0.998 	 P(Y^(A = Asian) = 0) =     0.504 	 ACE =   0.494
